In [47]:
from cntk.io import StreamDef, StreamDefs, MinibatchSource, CTFDeserializer, INFINITELY_REPEAT

labels_stream = StreamDef(field='labels', shape=3, is_sparse=False)
features_stream = StreamDef(field='features', shape=4, is_sparse=False)

deserializer = CTFDeserializer('iris.ctf', StreamDefs(labels=labels_stream, features=features_stream))

minibatch_source = MinibatchSource(deserializer, randomize=True, max_sweeps=INFINITELY_REPEAT)

In [48]:
from cntk import default_options, input_variable
from cntk.layers import Dense, Sequential
from cntk.ops import log_softmax, relu, sigmoid

model = Sequential([
    Dense(4, activation=sigmoid),
    Dense(3, activation=log_softmax)
])

features = input_variable(4)
labels = input_variable(3)

z = model(features)

In [49]:
from cntk.losses import cross_entropy_with_softmax
from cntk.learners import sgd 

loss = cross_entropy_with_softmax(z, labels)
learner = sgd(z.parameters, 0.1)

In [50]:
from cntk.logging import ProgressPrinter
from cntk.train import Trainer, training_session

minibatch_size = 16
samples_per_epoch = 150
num_epochs = 30
max_samples = samples_per_epoch * num_epochs

input_map = {
    features: minibatch_source.streams.features,
    labels: minibatch_source.streams.labels
}

progress_writer = ProgressPrinter(0)
trainer = Trainer(z, (loss, None), learner, progress_writer)

session = training_session(trainer, 
                           mb_source=minibatch_source,
                           mb_size=minibatch_size, 
                           model_inputs_to_streams=input_map, 
                           max_samples=max_samples)

session.train()

 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Learning rate per minibatch: 0.1
     1.53       1.53          0          0            16
     1.27       1.14          0          0            48
     1.23       1.21          0          0           112
     1.15       1.08          0          0           240
      1.1       1.04          0          0           496
     1.02      0.953          0          0          1008
    0.913      0.805          0          0          2032
    0.736      0.559          0          0          4080


In [51]:
CTFDeserializer??